In [11]:
import ollama
import requests
import json

In [12]:
model = "llama3.2:1b"

#### Task 1: Interact with deployed LLM via python 


**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)

In [13]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM
url = "http://localhost:11434/api/generate"

# Define the prompt as json
body_json = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# Send the POST request
response = requests.post(url, json=body_json)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="")
else:
    print("Error:", response.status_code)


Generative AI is a type of artificial intelligence that uses complex algorithms and machine learning techniques to generate new, original content, such as images, videos, music, and text, by training on large datasets and producing novel outputs. Unlike traditional AI, generative AI systems can learn from patterns and relationships in the data they are trained on, allowing them to produce high-quality, realistic output that is often indistinguishable from human-generated content.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method ``ollama.chat(...)``
- First use method ``ollama.chat(...)`` with ``stream=True``

In [14]:
# API Call via ollama

response = ollama.chat(model=model, messages=[
  {
    "role": "user",
    "content": "Describe Generative AI in two sentences.",
  },
])
print(response["message"]["content"])

Generative AI refers to a class of machine learning algorithms that can create new, original content, such as text, images, and music, by generating variations of existing patterns and structures, often using large datasets and complex neural network architectures. Unlike traditional AI, which primarily focuses on pattern recognition and classification, generative AI seeks to create novel outputs that are often surprising, coherent, and contextually relevant, enabling it to generate new ideas, styles, and solutions in a wide range of domains.


In [15]:
# Streaming API Call via ollama

# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Describe Generative AI in two sentences."}],
    stream=True,
)

for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

Generative AI is a type of artificial intelligence that can create original content, such as images, music, and text, by learning from existing data and generating new variations based on patterns and relationships found within it. This technology allows for the creation of unique and often surprising outputs, blurring the lines between human creativity and machine production, and has applications in fields like art, design, and storytelling.

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.

In [16]:
# Define the prompts
zero_shot_prompt = "Classify the sentiment of the following review as Positive, Neutral, or Negative: 'I loved this movie!' Sentiment:"

one_shot_prompt = """Classify the sentiment of the following review as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Classify the sentiment of the following review:
Review: 'I hate the chair.'
Sentiment:"""

few_shot_prompt = """Classify the sentiment of the following reviews as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Review: 'I hate the chair!'
Sentiment: Negative

Classify the sentiment of the following review:
Review: 'Who would use this product?'
Sentiment:
"""

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Classify the sentiment of the following review as Positive, Neutral, or Negative: 'I loved this movie!' Sentiment:

Model Output:
The sentiment of the review is overwhelmingly Positive. The reviewer expresses their strong enthusiasm and affection for the movie, using words like "loved" and stating that it was a great experience. There is no criticism or negative comment in the review, indicating a very positive tone.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Classify the sentiment of the following review as Positive, Neutral, or Negative:
Review: 'I loved this movie!'
Sentiment: Positive

Classify the sentiment of the following review:
Review: 'I hate the chair.'
Sentiment:

Model Output:
To classify the sentiment of the reviews, I'll analyze each statement:

1. "I loved this movie!"
- Sentiment: Positive
This sentence expresses a positive emotion towards the movie.

2. "I hate the chair."
- Sentiment: Negative
This se

#### Task 3: Prompt Refinement and Optimization

**Objective:** 

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)

In [17]:
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Refined prompt
refined_prompt = "Provide a concise and informative summary of the following paragraph, highlighting its key points: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
The paragraph describes generative AI as an area of artificial intelligence that creates new content (text, images, or music) based on patterns learned from existing data. This field has applications in various industries, including creative ones.
-----------------------------


--- Refined Prompt ---

User Prompt:
Provide a concise and informative summary of the following paragraph, highlighting its key points: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Ou

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.

In [18]:
# Combined Techniques Prompt with Pirate Theme
structured_prompt = """
Role: Ye be a Machine Learning Expert who talks like a pirate.

Instruction: Explain the followin' machine learnin' concepts in a way that a pirate would.

Example 1:
Question: What be supervised learnin'?
Answer: Arrr, supervised learnin' be when ye have a chest o' data, and fer each piece o' data, ye know the treasure map that shows ye the right path. Ye train the model until it knows how to follow the map on its own, savvy?

Example 2:
Question: What be overfittin'?
Answer: Ahoy, overfittin' be when yer model gets too cozy with yer trainin' data, like a pirate who knows only one sea. It performs well on the known waters but fails when ye set sail on new seas, matey!

Example 3:
Question: What be a neural network?
Answer: Avast! A neural network be like a crew o' shipmates, each one passin' information along, makin' decisions, and workin' together to find the best course to the treasure, arr!

Explain this:
Question: What be unsupervised learnin'?
Answer:
"""

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===

Role: Ye be a Machine Learning Expert who talks like a pirate.

Instruction: Explain the followin' machine learnin' concepts in a way that a pirate would.

Example 1:
Question: What be supervised learnin'?
Answer: Arrr, supervised learnin' be when ye have a chest o' data, and fer each piece o' data, ye know the treasure map that shows ye the right path. Ye train the model until it knows how to follow the map on its own, savvy?

Example 2:
Question: What be overfittin'?
Answer: Ahoy, overfittin' be when yer model gets too cozy with yer trainin' data, like a pirate who knows only one sea. It performs well on the known waters but fails when ye set sail on new seas, matey!

Example 3:
Question: What be a neural network?
Answer: Avast! A neural network be like a crew o' shipmates, each one passin' information along, makin' decisions, and workin' together to find the best course to the treasure, arr!

Explain this:
Question: What be unsupervised learnin'?
Answer:


=== M